In [1]:
import chess.pgn
import pandas as pd
import chess
import numpy as np
from scipy import linalg

In [2]:
ls -shr

total 90M
 16K README.md                              568K chess.ipynb
 89M lichess_db_standard_rated_2013-01.pgn   20K chess-and-eigenvectors.ipynb


In [3]:
board = chess.Board()

In [4]:
pgn = open('lichess_db_standard_rated_2013-01.pgn',encoding="utf-8")

In [5]:
#creates a list with one entry for each space on the board
spaces = []
for i in range(1,9):
    for j in range (1,9):
        spaces.append(chr(j+96)+str(i))

In [6]:
#create a matrix to tally weights for every pair of opposing pieces
pieces_list = [space for space in spaces if any(map(space.__contains__, ['1','2','7','8']))]
weights = pd.DataFrame(0,index=pieces_list, columns=pieces_list)

In [7]:
#create a state dictionary of each square and the initial square of each piece
#Those 'None' entries are the empty squares in the middle of the board, before the first move
#Each piece is on it's own square to begin
state_dict = {space:(space  if any(map(space.__contains__, ['1','2','7','8'])) else None) for space  in spaces}
state_dict

{'a1': 'a1',
 'b1': 'b1',
 'c1': 'c1',
 'd1': 'd1',
 'e1': 'e1',
 'f1': 'f1',
 'g1': 'g1',
 'h1': 'h1',
 'a2': 'a2',
 'b2': 'b2',
 'c2': 'c2',
 'd2': 'd2',
 'e2': 'e2',
 'f2': 'f2',
 'g2': 'g2',
 'h2': 'h2',
 'a3': None,
 'b3': None,
 'c3': None,
 'd3': None,
 'e3': None,
 'f3': None,
 'g3': None,
 'h3': None,
 'a4': None,
 'b4': None,
 'c4': None,
 'd4': None,
 'e4': None,
 'f4': None,
 'g4': None,
 'h4': None,
 'a5': None,
 'b5': None,
 'c5': None,
 'd5': None,
 'e5': None,
 'f5': None,
 'g5': None,
 'h5': None,
 'a6': None,
 'b6': None,
 'c6': None,
 'd6': None,
 'e6': None,
 'f6': None,
 'g6': None,
 'h6': None,
 'a7': 'a7',
 'b7': 'b7',
 'c7': 'c7',
 'd7': 'd7',
 'e7': 'e7',
 'f7': 'f7',
 'g7': 'g7',
 'h7': 'h7',
 'a8': 'a8',
 'b8': 'b8',
 'c8': 'c8',
 'd8': 'd8',
 'e8': 'e8',
 'f8': 'f8',
 'g8': 'g8',
 'h8': 'h8'}

In [11]:
game_counter = 0
total_move_counter = 0
game = chess.pgn.read_game(pgn)

while game is not None:
    counter = 0
    state_dict = {space:(space  if any(map(space.__contains__, ['1','2','7','8'])) else None) for space  in spaces}

    #print(game)
    #do stuff here



    #for each line in the file
    for move in game.mainline_moves():
        counter += 1
        if board.is_pseudo_legal(move):


            if board.is_capture(move):
                capturing_piece = state_dict[chess.square_name(move.from_square)]
                captured_piece = state_dict[chess.square_name(move.to_square)]
                #print('capturing piece',capturing_piece)
                #print('captured piece',captured_piece)

                if board.is_en_passant(move):
                    #do en passant stuff here
                    print("en passant")
                elif capturing_piece is not None:
                    if captured_piece is not None:

                        #record the kill in a matrix of all kills
                        #print(counter,chess.square_name(move.from_square), "takes on",chess.square_name(move.to_square))
                        #print(counter,state_dict[chess.square_name(move.from_square)], "takes",state_dict[chess.square_name(move.to_square)])

                        #what piece is there now?
                     

                        #increment an entry in the weights matrix when one piece takes another
                        weights.loc[state_dict[chess.square_name(move.from_square)],state_dict[chess.square_name(move.to_square)]] += 1

                        #update the piece on the target square
                        state_dict[chess.square_name(move.to_square)] = state_dict[chess.square_name(move.from_square)]

                        #update the source square to be empty
                        state_dict[chess.square_name(move.from_square)] = None

                        #board.push(move)  # Make the move


                        #if df.loc[i+1,'move_no'] == 1:
                        #    state_dict = {space:(space  if any(map(space.__contains__, ['1','2','7','8'])) else None) for space  in spaces}

            else:


                #add castling special cases to record both moves
                if board.is_kingside_castling(move):
                    if board.turn:
                        #print(counter,'... and white castles kingside')
                        #move the appopriate rook
                        state_dict['f1'] = 'h1'
                        #leaving the rook source square empty
                        state_dict['h1'] = 'None'
                        #board.push(move)  # Make the move



                    elif not board.turn:
                        #print(counter,'... and black castles kingside')
                        #move the appopriate rook
                        state_dict['f8'] = 'h8'
                        #leaving the rook source square empty
                        state_dict['h8'] = 'None'
                        #board.push(move)  # Make the move

                elif board.is_queenside_castling(move):
                    if board.turn:
                        #print(counter,'... and white castles queenside')
                        #move the appopriate rook
                        state_dict['d1'] = 'a1'
                        #leaving the rook source square empty
                        state_dict['a1'] = 'None'
                        #board.push(move)  # Make the move

                    elif not board.turn:
                        #print(counter,'... and black castles queenside')
                        #move the appopriate rook
                        state_dict['d8'] = 'a8'
                        #leaving the rook source square empty
                        state_dict['a8'] = 'None'
                        #board.push(move)  # Make the move

                else:
                    #update the piece on the target square
                    state_dict[chess.square_name(move.to_square)] = state_dict[chess.square_name(move.from_square)]

                    #update the source square to be empty
                    state_dict[chess.square_name(move.from_square)] = None


                    #print(counter, chess.square_name(move.from_square),chess.square_name(move.to_square))

        board.push(move)  # Make the move
        #print(state_dict)
            
    if board.is_checkmate():
        checkers_list = list(board.checkers())
        checkers_list = [chess.square_name(x) for x in checkers_list]
        print('checkmate')
        print(checkers_list)
    game_counter += 1
    #print('end of game',game_counter)
    
    total_move_counter += counter
    #print('total moves analyzed',total_move_counter)

    
    #print('board outcome',board.outcome())


    #print('is game over?',board.is_game_over())
    board.reset()

    game = chess.pgn.read_game(pgn)




    #print(state_dict)

    

checkmate
['f7']
checkmate
['g2']
checkmate
['d8']
checkmate
['e4']
checkmate
['a6']
checkmate
['c6']
checkmate
['b1']
checkmate
['f2']
checkmate
['g7']
checkmate
['a1']
checkmate
['e1']
checkmate
['f7']
checkmate
['h4']
checkmate
['d6']
checkmate
['d1']
checkmate
['b2']
checkmate
['g4']
checkmate
['b7']
checkmate
['h1']
checkmate
['f7']
checkmate
['h5']
en passant
checkmate
['b4']
checkmate
['g7']
checkmate
['e1']
checkmate
['f7']
checkmate
['g7']
en passant
checkmate
['c7']
checkmate
['h7']
checkmate
['d4']
checkmate
['e3']
en passant
checkmate
['g2']
checkmate
['c1']
checkmate
['h8']
checkmate
['g2']
checkmate
['c2']
checkmate
['d2']
checkmate
['c7']
en passant
checkmate
['e8']
checkmate
['e7']
checkmate
['b2']
checkmate
['f2']
en passant
checkmate
['e8']
checkmate
['h1']
checkmate
['h8']
checkmate
['c4']
checkmate
['d8']
checkmate
['f3']
checkmate
['h8']
checkmate
['c8']
checkmate
['h7']
checkmate
['c1']
checkmate
['g3']
checkmate
['f2']
checkmate
['h2']
checkmate
['f6']
checkmate


KeyboardInterrupt: 

In [9]:
print(game_counter,'games analyzed')
print(total_move_counter,'moves analyzed')

3176 games analyzed
209134 moves analyzed


In [10]:
weights

,a1,b1,c1,d1,e1,f1,g1,h1,a2,b2,...,g7,h7,a8,b8,c8,d8,e8,f8,g8,h8
a1,0,2,2,0,0,0,1,3,6,2,...,71,62,314,97,104,125,0,98,82,259
b1,1,0,2,0,0,1,1,2,3,2,...,39,34,102,134,153,116,0,241,347,86
c1,5,3,0,3,0,1,2,0,4,1,...,114,89,66,241,0,104,0,554,531,132
d1,3,1,0,0,0,3,2,2,4,3,...,167,173,238,329,410,708,0,298,300,215
e1,0,0,0,0,0,0,0,0,0,0,...,13,23,21,26,32,66,0,46,30,31
f1,2,3,2,3,0,0,2,3,0,1,...,47,77,112,469,563,92,0,0,263,70
g1,2,0,3,0,0,0,0,0,1,1,...,75,59,63,409,210,119,0,171,159,85
h1,2,3,2,2,0,0,1,0,5,4,...,95,99,254,96,144,152,0,144,112,299
a2,1,0,1,1,0,0,1,0,0,2,...,2,5,12,30,24,13,0,18,10,8
b2,1,2,1,1,0,0,0,0,1,0,...,1,0,12,46,13,17,0,186,55,3


In [ ]:
a = weights - np.identity(32)
print(a)

In [ ]:
b = np.zeros(32)
print(b)

In [ ]:
x = linalg.solve(a, b)
x

### Why doesn't it work?
### Because the solver is finding the trivial solution. i.e. just a vector full of zeroes.
### To check that there is actually a non trivial solution, the determinant of A must be non-zero.

In [ ]:
np.linalg.det(a)

### OK since the determinant is not zero, there must be at least one non-trivial solution. How to find it?
### An homogenous linear equation can be solved with a Matrix Factorization.

In [ ]:
eigen_values, eigen_vectors = np.linalg.eig(a)  

In [ ]:
values = eigen_vectors[:, np.argmin(eigen_values)] 

In [ ]:
## Let's normalize the values

In [ ]:
list = []
for i in values:
    list.append(abs(float(i)))

In [ ]:
#The fractional inverse of the strongest pawn
factor = 1 / np.max(list[8:-8])

In [ ]:
normed = [i*factor for i in list]

In [ ]:
normalized_list = ["%.2f"%i for i in normed]

In [ ]:
print(normalized_list[0:8])
print(normalized_list[8:8+8])
print(normalized_list[8+8:8+8+8])
print(normalized_list[-8:])

In [ ]:
board.reset()
game = chess.pgn.read_game(pgn)

for move in game.mainline_moves():
        print(move.uci())
        print('turn?' , board.turn)
        print('is game over?' , board.is_game_over())
        print('from square: ' ,chess.square_name(move.from_square))
        print('to square: ' ,chess.square_name(move.to_square))
        print('drop: ' ,move.drop)
        print('is capture?' , board.is_capture(move))
        print('is en passant capture?' , board.is_en_passant(move))
        print('is kingside castle?' , board.is_kingside_castling(move))
        print('is quenside castle?' , board.is_queenside_castling(move))

        board.push(move)  # Make the move
        print('is_checkmate?' , board.is_checkmate())
        #print('checkers' , board.checkers())
        print('outcome' , board.outcome())

        #print(board)
        #print(board.piece_map())
    #board